In [29]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [30]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [31]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [32]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [33]:
# Choose a cutoff value and create a list of classifications to be replaced...Choosing 1000
#choosing a different value of 2000 for optimization, compared to initial attempt in funding_decision.ipynb
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [34]:
# Convert categorical data to numeric with `pd.get_dummies`
columns_to_encode = ['APPLICATION_TYPE',	'AFFILIATION', 'CLASSIFICATION', 'INCOME_AMT', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS']
app_dummies = pd.get_dummies(application_df, columns=columns_to_encode, dtype=int)
app_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0


In [35]:
# Split our preprocessed data into our features and target arrays
X = app_dummies.drop(['IS_SUCCESSFUL'], axis=1).values
y = app_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Attempt 1: Use more hidden layers

In [37]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 15
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
#fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                4160      
                                                                 
 dense_9 (Dense)             (None, 30)                2430      
                                                                 
 dense_10 (Dense)            (None, 15)                465       
                                                                 
 dense_11 (Dense)            (None, 15)                240       
                                                                 
 dense_12 (Dense)            (None, 15)                240       
                                                                 
 dense_13 (Dense)            (None, 1)                 16        
                                                                 
Total params: 7551 (29.50 KB)
Trainable params: 7551 (

In [38]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the second model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 2ms/step - loss: 0.5712 - accuracy: 0.7235
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5535 - accuracy: 0.7305
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7326
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7312
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5470 - accuracy: 0.7331
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5465 - accuracy: 0.7329
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5460 - accuracy: 0.7343
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5455 - accura

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5637 - accuracy: 0.7282 - 581ms/epoch - 2ms/step
Loss: 0.5636518597602844, Accuracy: 0.7281632423400879


In [41]:
# Export our model to HDF5 file  Per tutor Justin: modelname.save('file name to save.h5')
nn.save('AlphabetSoupCharity_1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Attempt 2: Use different activation functions, trying gelu:

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="gelu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="gelu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 80)                4160      
                                                                 
 dense_15 (Dense)            (None, 30)                2430      
                                                                 
 dense_16 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6621 (25.86 KB)
Trainable params: 6621 (25.86 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [44]:
# Train the second model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5753 - accuracy: 0.7201
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5557 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5499 - accuracy: 0.7308
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7330
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5468 - accuracy: 0.7333
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5455 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accura

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5583 - accuracy: 0.7277 - 678ms/epoch - 3ms/step
Loss: 0.5582723021507263, Accuracy: 0.7276967763900757


In [46]:
# Export our model to HDF5 file  Per Justin: modelname.save('file name to save.h5')
nn.save('AlphabetSoupCharity_2.h5')

Optimization Attempt 3: remove APPLICATION_TYPE from consideration

In [47]:
# Attempt dropping Application_type to see if that helps optimization (attempt 3)
application_df = application_df.drop(['APPLICATION_TYPE'], axis=1)
application_df

,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [49]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [50]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [51]:
# Convert categorical data to numeric with `pd.get_dummies`
columns_to_encode = ['CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'AFFILIATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT']
app_dummies = pd.get_dummies(application_df, columns=columns_to_encode, dtype=int)
app_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,...,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [52]:
app_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   IS_SUCCESSFUL                 34299 non-null  int64
 3   CLASSIFICATION_C1000          34299 non-null  int64
 4   CLASSIFICATION_C1200          34299 non-null  int64
 5   CLASSIFICATION_C2000          34299 non-null  int64
 6   CLASSIFICATION_C2100          34299 non-null  int64
 7   CLASSIFICATION_C3000          34299 non-null  int64
 8   CLASSIFICATION_Other          34299 non-null  int64
 9   USE_CASE_CommunityServ        34299 non-null  int64
 10  USE_CASE_Heathcare            34299 non-null  int64
 11  USE_CASE_Other                34299 non-null  int64
 12  USE_CASE_Preservation         34299 non-null  int64
 13  USE_CASE_ProductDev           3

In [53]:
# Split our preprocessed data into our features and target arrays
X = app_dummies.drop(['IS_SUCCESSFUL'], axis=1).values
y = app_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [54]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [55]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 80)                2800      
                                                                 
 dense_18 (Dense)            (None, 30)                2430      
                                                                 
 dense_19 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5261 (20.55 KB)
Trainable params: 5261 (20.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [57]:
# Train the second model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5953 - accuracy: 0.6967
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5831 - accuracy: 0.7041
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5795 - accuracy: 0.7071
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5785 - accuracy: 0.7090
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5774 - accuracy: 0.7091
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5774 - accuracy: 0.7088
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5762 - accuracy: 0.7089
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5759 - accuracy: 0.7088
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5757 - accuracy: 0.7100
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5745 - accura

In [64]:
# Evaluate the model using the test data.  The accuracy is worse.
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5787 - accuracy: 0.7099 - 424ms/epoch - 2ms/step
Loss: 0.5786702036857605, Accuracy: 0.709854245185852


In [65]:
# Export our model to HDF5 file  Per Justin: modelname.save('file name to save.h5')
nn.save('AlphabetSoupCharity_3.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
